## Layer 3: Data Preparation

This notebook prepares the raw UIDAI Aadhaar datasets for analysis.  
The objective of this layer is to clean, standardise, aggregate, and integrate enrolment, demographic update, and biometric update data into a single analysis-ready dataset.


In [ ]:
import pandas as pd
import numpy as np

### Step 1: Dataset Ingestion

The UIDAI datasets are provided as ZIP files containing multiple CSV parts.  
All CSV files belonging to the same dataset are extracted and combined to form a unified table for each dataset.


In [ ]:
# Unzip all datasets
import zipfile
import os

BASE_PATH = '/content/UIDAI_Datasets'

def unzip(zip_name, extract_folder):
    zip_path = os.path.join(BASE_PATH, zip_name)
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(extract_folder)

unzip('api_data_aadhar_enrolment.zip', '/content/enrolment')
unzip('api_data_aadhar_demographic.zip', '/content/demographic')
unzip('api_data_aadhar_biometric.zip', '/content/biometric')

In [ ]:
os.listdir('/content/enrolment')
os.listdir('/content/demographic')
os.listdir('/content/biometric')

['api_data_aadhar_biometric']

In [ ]:
# Read and combine Enrolment dataset

enrolment_path = '/content/enrolment/api_data_aadhar_enrolment'

enrolment_files = [
    os.path.join(enrolment_path, f)
    for f in os.listdir(enrolment_path)
    if f.endswith('.csv')
]

enrolment_df = pd.concat(
    [pd.read_csv(f) for f in enrolment_files],
    ignore_index=True
)

In [ ]:
enrolment_df.shape

(1006029, 7)

In [ ]:
enrolment_df.head()

,date,state,district,pincode,age_0_5,age_5_17,age_18_greater
0,31-12-2025,Karnataka,Bidar,585330,2,3,0
1,31-12-2025,Karnataka,Bidar,585402,6,0,0
2,31-12-2025,Karnataka,Bidar,585413,1,0,0
3,31-12-2025,Karnataka,Bidar,585418,1,2,0
4,31-12-2025,Karnataka,Bidar,585421,4,3,0


In [ ]:
# Read and combine Demographic dataset

demographic_path = '/content/demographic/api_data_aadhar_demographic'

demographic_files = [
    os.path.join(demographic_path, f)
    for f in os.listdir(demographic_path)
    if f.endswith('.csv')
]

demographic_df = pd.concat(
    [pd.read_csv(f) for f in demographic_files],
    ignore_index=True
)

In [ ]:
demographic_df.shape

(2071700, 6)

In [ ]:
demographic_df.head()

,date,state,district,pincode,demo_age_5_17,demo_age_17_
0,18-11-2025,Karnataka,Hasan,573118,0,2
1,18-11-2025,Karnataka,Hasan,573124,1,3
2,18-11-2025,Karnataka,Hasan,573150,0,2
3,18-11-2025,Karnataka,Hassan,573113,0,1
4,18-11-2025,Karnataka,Hassan,573120,1,4


In [ ]:
# Read and combine Biometric dataset

biometric_path = '/content/biometric/api_data_aadhar_biometric'

biometric_files = [
    os.path.join(biometric_path, f)
    for f in os.listdir(biometric_path)
    if f.endswith('.csv')
]

biometric_df = pd.concat(
    [pd.read_csv(f) for f in biometric_files],
    ignore_index=True
)

In [ ]:
biometric_df.shape

(1861108, 6)

In [ ]:
biometric_df.head()

,date,state,district,pincode,bio_age_5_17,bio_age_17_
0,07-11-2025,Haryana,Yamuna Nagar,135002,4,6
1,07-11-2025,Haryana,Yamunanagar,135001,1,2
2,07-11-2025,Himachal Pradesh,Bilaspur,174004,2,2
3,07-11-2025,Himachal Pradesh,Bilaspur,174005,1,0
4,07-11-2025,Himachal Pradesh,Bilaspur,174013,3,1


### Step 2: Date Standardisation

All date fields are converted from string format (DD-MM-YYYY) to datetime objects.  
This ensures consistent temporal analysis and prevents silent parsing errors.


In [ ]:
# Convert date column to datetime

enrolment_df['date'] = pd.to_datetime(
    enrolment_df['date'],
    format='%d-%m-%Y',
    errors='raise'
)

demographic_df['date'] = pd.to_datetime(
    demographic_df['date'],
    format='%d-%m-%Y',
    errors='raise'
)

biometric_df['date'] = pd.to_datetime(
    biometric_df['date'],
    format='%d-%m-%Y',
    errors='raise'
)

In [ ]:
enrolment_df['date'].min(), enrolment_df['date'].max()

(Timestamp('2025-03-02 00:00:00'), Timestamp('2025-12-31 00:00:00'))

In [ ]:
demographic_df['date'].min(), demographic_df['date'].max()

(Timestamp('2025-03-01 00:00:00'), Timestamp('2025-12-29 00:00:00'))

In [ ]:
biometric_df['date'].min(), biometric_df['date'].max()

(Timestamp('2025-03-01 00:00:00'), Timestamp('2025-12-29 00:00:00'))

### Step 3: Monthly Time Bucketing

Daily records are mapped to monthly buckets to align the analysis with governance-level decision making.  
Each record is assigned to the first day of its corresponding month.


In [ ]:
# Create month column from date

enrolment_df['month'] = enrolment_df['date'].dt.to_period('M').dt.to_timestamp()
demographic_df['month'] = demographic_df['date'].dt.to_period('M').dt.to_timestamp()
biometric_df['month'] = biometric_df['date'].dt.to_period('M').dt.to_timestamp()

In [ ]:
enrolment_df[['date', 'month']].head()

,date,month
0,2025-12-31,2025-12-01
1,2025-12-31,2025-12-01
2,2025-12-31,2025-12-01
3,2025-12-31,2025-12-01
4,2025-12-31,2025-12-01


In [ ]:
demographic_df[['date', 'month']].head()

,date,month
0,2025-11-18,2025-11-01
1,2025-11-18,2025-11-01
2,2025-11-18,2025-11-01
3,2025-11-18,2025-11-01
4,2025-11-18,2025-11-01


In [ ]:
biometric_df[['date', 'month']].head()

,date,month
0,2025-11-07,2025-11-01
1,2025-11-07,2025-11-01
2,2025-11-07,2025-11-01
3,2025-11-07,2025-11-01
4,2025-11-07,2025-11-01


### Step 4: Scope Discipline and Column Selection

Only columns relevant to the analytical objective are retained.  
The analysis is intentionally restricted to month, state, district, and age-group counts to avoid scope creep.


In [ ]:
# Select required columns – Enrolment

enrolment_df = enrolment_df[
    ['month', 'state', 'district', 'age_0_5', 'age_5_17', 'age_18_greater']
]

In [ ]:
enrolment_df.columns

Index(['month', 'state', 'district', 'age_0_5', 'age_5_17', 'age_18_greater'], dtype='object')

In [ ]:
# Select required columns – Demographic

demographic_df = demographic_df[
    ['month', 'state', 'district', 'demo_age_5_17', 'demo_age_17_']
]

In [ ]:
demographic_df.columns

Index(['month', 'state', 'district', 'demo_age_5_17', 'demo_age_17_'], dtype='object')

In [ ]:
# Select required columns – Biometric

biometric_df = biometric_df[
    ['month', 'state', 'district', 'bio_age_5_17', 'bio_age_17_']
]

In [ ]:
biometric_df.columns

Index(['month', 'state', 'district', 'bio_age_5_17', 'bio_age_17_'], dtype='object')

### Step 5: Monthly Aggregation

All datasets are aggregated at the month × state × district level.  
This step converts raw transaction-level data into structured indicators suitable for system-level analysis.


In [ ]:
# Monthly aggregation – Enrolment

enrolment_monthly = (
    enrolment_df
    .groupby(['month', 'state', 'district'], as_index=False)
    .sum()
)

In [ ]:
enrolment_monthly.head()

,month,state,district,age_0_5,age_5_17,age_18_greater
0,2025-03-01,Andhra Pradesh,Spsr Nellore,43,44,29
1,2025-03-01,Assam,Baksa,11,14,13
2,2025-03-01,Assam,Barpeta,24,34,10
3,2025-03-01,Assam,Bongaigaon,25,33,31
4,2025-03-01,Assam,Chirang,12,29,15


In [ ]:
# Monthly aggregation – Demographic updates

demographic_monthly = (
    demographic_df
    .groupby(['month', 'state', 'district'], as_index=False)
    .sum()
)

In [ ]:
demographic_monthly.head()

,month,state,district,demo_age_5_17,demo_age_17_
0,2025-03-01,Andaman and Nicobar Islands,Nicobar,32,360
1,2025-03-01,Andaman and Nicobar Islands,North And Middle Andaman,20,402
2,2025-03-01,Andaman and Nicobar Islands,South Andaman,74,450
3,2025-03-01,Andhra Pradesh,Adilabad,390,3950
4,2025-03-01,Andhra Pradesh,Alluri Sitharama Raju,507,4448


In [ ]:
# Monthly aggregation – Biometric updates

biometric_monthly = (
    biometric_df
    .groupby(['month', 'state', 'district'], as_index=False)
    .sum()
)

In [ ]:
biometric_monthly.head()

,month,state,district,bio_age_5_17,bio_age_17_
0,2025-03-01,Andaman & Nicobar Islands,Andamans,16,193
1,2025-03-01,Andaman and Nicobar Islands,Nicobar,178,101
2,2025-03-01,Andaman and Nicobar Islands,North And Middle Andaman,470,347
3,2025-03-01,Andaman and Nicobar Islands,South Andaman,948,450
4,2025-03-01,Andhra Pradesh,Adilabad,897,4366


In [ ]:
# Checking aggregated table month count
enrolment_df['month'].nunique()

9

In [ ]:
# Observing all distinct months in enrolment data
sorted(enrolment_df['month'].unique())

[Timestamp('2025-03-01 00:00:00'),
 Timestamp('2025-04-01 00:00:00'),
 Timestamp('2025-05-01 00:00:00'),
 Timestamp('2025-06-01 00:00:00'),
 Timestamp('2025-07-01 00:00:00'),
 Timestamp('2025-09-01 00:00:00'),
 Timestamp('2025-10-01 00:00:00'),
 Timestamp('2025-11-01 00:00:00'),
 Timestamp('2025-12-01 00:00:00')]

In [ ]:
demographic_df['month'].nunique()

9

In [ ]:
biometric_df['month'].nunique()

9

### Step 6: Dataset Integration

The aggregated enrolment, demographic update, and biometric update datasets are merged into a single table using left joins.  
This ensures enrolment remains the base population while preserving update information.


In [ ]:
# Join monthly enrolment, demographic, and biometric tables

identity_metrics = (
    enrolment_monthly
    .merge(
        demographic_monthly,
        on=['month', 'state', 'district'],
        how='left'
    )
    .merge(
        biometric_monthly,
        on=['month', 'state', 'district'],
        how='left'
    )
)

In [ ]:
identity_metrics.describe()

,month,age_0_5,age_5_17,age_18_greater,demo_age_5_17,demo_age_17_,bio_age_5_17,bio_age_17_
count,5062,5062.000000,5062.000000,5062.000000,4376.000000,4376.000000,4954.000000,4954.000000
mean,2025-09-13 06:24:53.480837632,700.704267,339.862505,33.258198,807.018282,7414.562157,4284.947719,4170.483650
min,2025-03-01 00:00:00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,2025-09-01 00:00:00,101.250000,24.000000,0.000000,105.000000,981.750000,398.000000,457.250000
50%,2025-10-01 00:00:00,424.000000,113.000000,7.000000,461.000000,3908.000000,2512.000000,2397.500000
75%,2025-11-01 00:00:00,980.500000,382.000000,26.000000,1087.000000,9211.500000,6084.500000,5850.750000
max,2025-12-01 00:00:00,12270.000000,6314.000000,2404.000000,9362.000000,131818.000000,51052.000000,59381.000000
std,NaN,861.515761,582.831904,103.892185,1016.001344,10555.567116,5282.686687,5488.577248


In [ ]:
identity_metrics.head()

,month,state,district,age_0_5,age_5_17,age_18_greater,demo_age_5_17,demo_age_17_,bio_age_5_17,bio_age_17_
0,2025-03-01,Andhra Pradesh,Spsr Nellore,43,44,29,NaN,NaN,NaN,NaN
1,2025-03-01,Assam,Baksa,11,14,13,397.0,4438.0,1333.0,527.0
2,2025-03-01,Assam,Barpeta,24,34,10,821.0,9181.0,8751.0,2053.0
3,2025-03-01,Assam,Bongaigaon,25,33,31,699.0,8681.0,3011.0,1170.0
4,2025-03-01,Assam,Chirang,12,29,15,179.0,1711.0,1611.0,679.0


In [ ]:
# Replacing NaNs with 0
identity_metrics.fillna(0, inplace=True)

In [ ]:
# Create total counts
# Total enrolments (all age groups)
identity_metrics['total_enrolments'] = (
    identity_metrics['age_0_5'] +
    identity_metrics['age_5_17'] +
    identity_metrics['age_18_greater']
)

# Total demographic updates
identity_metrics['total_demo_updates'] = (
    identity_metrics['demo_age_5_17'] +
    identity_metrics['demo_age_17_']
)

# Total biometric updates
identity_metrics['total_bio_updates'] = (
    identity_metrics['bio_age_5_17'] +
    identity_metrics['bio_age_17_']
)

In [ ]:
identity_metrics[
    ['total_enrolments', 'total_demo_updates', 'total_bio_updates']
].head()

,total_enrolments,total_demo_updates,total_bio_updates
0,116,0.0,0.0
1,38,4835.0,1860.0
2,68,10002.0,10804.0
3,89,9380.0,4181.0
4,56,1890.0,2290.0


### Step 7: Derived Metrics

Total enrolments, demographic updates, and biometric updates are computed.  
Update-to-enrolment ratios are derived to quantify Aadhaar update pressure across regions and time.


In [ ]:
# Compute update-to-enrolment ratios
# Demographic update ratio
identity_metrics['demo_update_ratio'] = (
    identity_metrics['total_demo_updates'] /
    identity_metrics['total_enrolments'].replace(0, np.nan)
)

# Biometric update ratio
identity_metrics['bio_update_ratio'] = (
    identity_metrics['total_bio_updates'] /
    identity_metrics['total_enrolments'].replace(0, np.nan)
)

In [ ]:
identity_metrics[
    ['demo_update_ratio', 'bio_update_ratio']
].describe()

,demo_update_ratio,bio_update_ratio
count,5062.000000,5062.000000
mean,11.412227,13.406626
std,41.500290,35.789282
min,0.000000,0.000000
25%,3.216551,3.603244
50%,5.976599,6.351876
75%,10.736147,12.186472
max,1599.297872,1101.255319


### Step 8: Export Analysis-Ready Datasets

The final aggregated datasets are exported as CSV files.  
These outputs serve as the fixed input for all subsequent analysis, indicator design, and visualisation layers.


In [ ]:
# Export final Layer 3 outputs

enrolment_monthly.to_csv('enrolment_monthly.csv', index=False)
demographic_monthly.to_csv('demographic_monthly.csv', index=False)
biometric_monthly.to_csv('biometric_monthly.csv', index=False)
identity_metrics.to_csv('identity_metrics.csv', index=False)